In [11]:
import pandas as pd
import pickle
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split,cross_validate,StratifiedKFold
from sklearn import svm
from keras.utils import to_categorical

In [13]:
X=pd.read_csv("data/X.csv", sep=' ',header=None, dtype=float)
X=X.values
Y=pd.read_csv("data/y_williams_vs_others.csv", header=None)
y = Y.values.ravel()
print(X.shape)
print(np.sum(y))

(13233, 4096)
52


In [127]:
print(X)
print(y)
y_cat = to_categorical(y)
classes = np.unique(y)
nClasses = len(classes)
print(nClasses)
print(y_cat)

[[0.10635443 0.09975202 0.11191927 ... 0.04983379 0.05402631 0.04627987]
 [0.         0.         0.         ... 0.00410788 0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.00074521 0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.00030042 0.         0.        ]]
[0 0 0 ... 0 0 0]
2
[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]


In [128]:
X_b = X.reshape(-1,64,64,1)
X_train, X_test, y_train, y_test = train_test_split(X_b, y, shuffle = True,stratify=y, test_size=1/3, random_state=7392)

In [129]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(8822, 64, 64, 1)
(4411, 64, 64, 1)
(8822,)
(4411,)


In [130]:
from keras import layers
from keras import models
from keras.layers.core import Activation
model = models.Sequential()
model.add(layers.Conv2D(64,(3,3),activation='relu',input_shape=(64,64,1)))
model.add(layers.MaxPooling2D(pool_size=(2, 2),strides=(2,2)))
model.add(layers.Conv2D(128, (3,3),activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2),strides=(2,2)))
#model.add(layers.Conv2D(64, (3,3),activation='relu'))
#model.add(layers.MaxPooling2D(pool_size=(2, 2),strides=(2,2)))
model.add(layers.Dropout(0.5))
model.add(layers.Flatten())

#model.add(layers.Dense(64, activation='tanh'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_40 (Conv2D)           (None, 62, 62, 64)        640       
_________________________________________________________________
max_pooling2d_40 (MaxPooling (None, 31, 31, 64)        0         
_________________________________________________________________
conv2d_41 (Conv2D)           (None, 29, 29, 128)       73856     
_________________________________________________________________
max_pooling2d_41 (MaxPooling (None, 14, 14, 128)       0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 14, 14, 128)       0         
_________________________________________________________________
flatten_17 (Flatten)         (None, 25088)             0         
_________________________________________________________________
dense_24 (Dense)             (None, 1)                 25089     
Total para

In [131]:
#train_X=X_train.reshape(-1,64,64,1)
model.fit(X_train,y_train,batch_size=64,epochs=10)

Epoch 1/10
8822/8822 [==============================] - 110s 12ms/step - loss: 0.0482 - acc: 0.9895
Epoch 2/10
8822/8822 [==============================] - 96s 11ms/step - loss: 0.0240 - acc: 0.9960
Epoch 3/10
8822/8822 [==============================] - 95s 11ms/step - loss: 0.0191 - acc: 0.9959
Epoch 4/10
8822/8822 [==============================] - 97s 11ms/step - loss: 0.0133 - acc: 0.9961
Epoch 5/10
8822/8822 [==============================] - 98s 11ms/step - loss: 0.0083 - acc: 0.9975
Epoch 6/10
8822/8822 [==============================] - 96s 11ms/step - loss: 0.0083 - acc: 0.9978
Epoch 7/10
8822/8822 [==============================] - 96s 11ms/step - loss: 0.0056 - acc: 0.9982
Epoch 8/10
8822/8822 [==============================] - 98s 11ms/step - loss: 0.0038 - acc: 0.9991
Epoch 9/10
8822/8822 [==============================] - 96s 11ms/step - loss: 0.0029 - acc: 0.9991
Epoch 10/10
8822/8822 [==============================] - 98s 11ms/step - loss: 0.0029 - acc: 0.9990


In [132]:
scores = model.evaluate(X_test, y_test)


4411/4411 [==============================] - 14s 3ms/step


In [133]:
print(scores)

[0.019700435756374857, 0.9975062344139651]


In [134]:
predict_function= model.predict_classes(X_test)
print(predict_function.shape)
print(y_test.shape)

(4411, 1)
(4411,)


In [135]:
from sklearn.metrics import f1_score
f1_williams=f1_score(y_test,predict_function)
print(f1_williams)

0.5217391304347826


In [136]:
predict_function= model.predict_classes(train_X)
print(predict_function.shape)
print(y_train.shape)

(8822, 1)
(8822,)


In [137]:
from sklearn.metrics import f1_score
f1_williams=f1_score(y_train,predict_function)
print(f1_williams)

0.923076923076923


In [138]:
model.save('williams.model')

In [139]:
from keras.models import load_model
model_final = load_model('williams.model')
model_final.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_40 (Conv2D)           (None, 62, 62, 64)        640       
_________________________________________________________________
max_pooling2d_40 (MaxPooling (None, 31, 31, 64)        0         
_________________________________________________________________
conv2d_41 (Conv2D)           (None, 29, 29, 128)       73856     
_________________________________________________________________
max_pooling2d_41 (MaxPooling (None, 14, 14, 128)       0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 14, 14, 128)       0         
_________________________________________________________________
flatten_17 (Flatten)         (None, 25088)             0         
_________________________________________________________________
dense_24 (Dense)             (None, 1)                 25089     
Total para

In [140]:
predict_function1 = model_final.predict_classes(train_X)
predict_function = model_final.predict_classes(test_X)
from sklearn.metrics import f1_score
f1_williams = f1_score(y_test,predict_function)
f1_williams1 = f1_score(y_train,predict_function1)
print(f1_williams1)
print(f1_williams)

0.923076923076923
0.5217391304347826


In [143]:
import pickle

In [144]:
p = [f1_williams1, f1_williams]
print (p)

[0.923076923076923, 0.5217391304347826]


In [145]:
pickle.dump(p, open('williams.pkl', 'wb'))